# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pycountry_convert as pc

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vac_df = pd.read_csv('../output_data/cities.csv') 
vac_df

,City,Country,Latitude,Longitude,Max_temp_F,Humidity_Percent,Cloudy_Percent,Wind_Speed_mph,Date
0,Jamestown,United States,42.10,-79.24,36.00,86,1,9.17,2020-12-10 23:43:38
1,Kapaa,United States,22.08,-159.32,82.40,69,1,6.93,2020-12-10 23:44:19
2,Fortuna,United States,40.60,-124.16,52.00,81,40,1.99,2020-12-10 23:45:35
3,Castro,Brazil,-24.79,-50.01,61.61,92,20,5.12,2020-12-10 23:42:38
4,San,Mali,13.30,-4.90,75.13,20,0,10.74,2020-12-10 23:44:11
...,...,...,...,...,...,...,...,...,...
555,Gautier,United States,30.39,-88.61,62.60,82,1,3.89,2020-12-10 23:47:40
556,Kangaatsiaq,Greenland,68.31,-53.46,16.79,91,99,10.20,2020-12-10 23:47:40
557,Cassilândia,Brazil,-19.11,-51.73,75.49,69,94,4.43,2020-12-10 23:47:40
558,Jaora,India,23.63,75.13,69.17,64,94,2.57,2020-12-10 23:47:41


In [3]:
vac_df.dtypes

City                 object
Country              object
Latitude            float64
Longitude           float64
Max_temp_F          float64
Humidity_Percent      int64
Cloudy_Percent        int64
Wind_Speed_mph      float64
Date                 object
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [6]:
# Store 'Lat' and 'Lng' into  locations 
locations = vac_df[["Latitude", "Longitude"]].astype(float)

# Weight
humheat = vac_df["Humidity_Percent"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humheat, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
# fig = gmaps.figure(layout=figure_layout)

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Vacation spots
vac_spot = (vac_df["Max_temp_F"] < 90) & (vac_df["Max_temp_F"] > 40) & (vac_df["Humidity_Percent"] < 60) & (vac_df["Country"] == "Australia")
vac_spot = vac_spot.to_frame('Destination')

vac_dest = vac_df.merge(vac_spot, how = "outer", left_index=True, right_index=True)
vac_dest["Destination"] = vac_dest["Destination"]*1
# vac_dest

vac_best = vac_dest.loc[vac_dest['Destination'] == 1]
vac_best

,City,Country,Latitude,Longitude,Max_temp_F,Humidity_Percent,Cloudy_Percent,Wind_Speed_mph,Date,Destination
69,Hobart,Australia,-42.88,147.33,64.0,47,75,12.75,2020-12-10 23:45:43,1
136,Cockburn,Australia,-32.08,141.00,69.8,28,69,18.34,2020-12-10 23:46:10,1
440,Narrabri,Australia,-30.32,149.78,75.2,47,90,11.41,2020-12-10 23:47:19,1
483,Innisfail,Australia,-17.53,146.03,79.0,59,61,5.99,2020-12-10 23:47:28,1
503,Mackay,Australia,-21.15,149.20,84.2,48,61,13.87,2020-12-10 23:47:31,1
527,Townsville,Australia,-19.25,146.80,84.2,58,75,13.87,2020-12-10 23:47:36,1
532,Ulladulla,Australia,-35.35,150.47,64.4,42,75,16.11,2020-12-10 23:47:37,1
542,Flinders,Australia,-34.58,150.86,64.4,42,75,16.11,2020-12-10 23:47:38,1


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = vac_best
del hotel_df['Cloudy_Percent']

hotel_df.rename(columns={'Latitude':'Lat', 'Longitude':'Lng',}, inplace=True)
hotel_df

C:\Users\troyy\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,City,Country,Lat,Lng,Max_temp_F,Humidity_Percent,Wind_Speed_mph,Date,Destination
69,Hobart,Australia,-42.88,147.33,64.0,47,12.75,2020-12-10 23:45:43,1
136,Cockburn,Australia,-32.08,141.00,69.8,28,18.34,2020-12-10 23:46:10,1
440,Narrabri,Australia,-30.32,149.78,75.2,47,11.41,2020-12-10 23:47:19,1
483,Innisfail,Australia,-17.53,146.03,79.0,59,5.99,2020-12-10 23:47:28,1
503,Mackay,Australia,-21.15,149.20,84.2,48,13.87,2020-12-10 23:47:31,1
527,Townsville,Australia,-19.25,146.80,84.2,58,13.87,2020-12-10 23:47:36,1
532,Ulladulla,Australia,-35.35,150.47,64.4,42,16.11,2020-12-10 23:47:37,1
542,Flinders,Australia,-34.58,150.86,64.4,42,16.11,2020-12-10 23:47:38,1


In [9]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

C:\Users\troyy\anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\troyy\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Country,Lat,Lng,Max_temp_F,Humidity_Percent,Wind_Speed_mph,Date,Destination,Hotel Name
69,Hobart,Australia,-42.88,147.33,64.0,47,12.75,2020-12-10 23:45:43,1,Vibe Hotel Hobart
136,Cockburn,Australia,-32.08,141.00,69.8,28,18.34,2020-12-10 23:46:10,1,Coburn Hotel
440,Narrabri,Australia,-30.32,149.78,75.2,47,11.41,2020-12-10 23:47:19,1,Kaputar Motel Narrabri
483,Innisfail,Australia,-17.53,146.03,79.0,59,5.99,2020-12-10 23:47:28,1,Tropical Lodge
503,Mackay,Australia,-21.15,149.20,84.2,48,13.87,2020-12-10 23:47:31,1,Mantra Mackay
527,Townsville,Australia,-19.25,146.80,84.2,58,13.87,2020-12-10 23:47:36,1,Mercure Townsville
532,Ulladulla,Australia,-35.35,150.47,64.4,42,16.11,2020-12-10 23:47:37,1,Beachhouse Mollymook
542,Flinders,Australia,-34.58,150.86,64.4,42,16.11,2020-12-10 23:47:38,1,Lakeview Hotel Motel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
locations

,Lat,Lng
69,-42.88,147.33
136,-32.08,141.00
440,-30.32,149.78
483,-17.53,146.03
503,-21.15,149.20
527,-19.25,146.80
532,-35.35,150.47
542,-34.58,150.86


In [16]:
# Create hotel layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2)

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel Name: {hotel}" for hotel in hotel_info]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))